In [ ]:
#https://towardsdatascience.com/azure-machine-learning-service-where-is-my-data-pjainani-86a77b93ab52

# Azure Machine Learning Service — Where is My Data

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()
ws

In [ ]:
blob_datastore_name='capstoneproject' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "azureml-blobstore-ab13107c-d7dd-4c18-a867-72c901b8efa5") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "mlstrg139721") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "On2xGZzfsR2/tlgwTHbwlTcocaONZC1UESaLBSDWGO/kJsxj5w31xlEDzt2nZiFPN9bbUwQfJeSjXzFgDSGfog==") # Storage account access key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [ ]:
# Get a named datastore from the current workspace
datastore = Datastore.get(ws,"capstoneproject")
datastore

In [ ]:
datastore = ws.get_default_datastore()
datastore

In [ ]:
# List all datastores registered in the current workspace
datastores = ws.datastores
for name, datastore in datastores.items():
    print(name, datastore.datastore_type)

In [ ]:
new_default_datastore="capstoneproject"
ws.set_default_datastore(new_default_datastore)

In [ ]:
# get the name of defult Datastore associated with the workspace.
default_dsname = ws.get_default_datastore().name
default_ds = ws.get_default_datastore()
print('default Datastore = ', default_dsname)

In [ ]:
default_ds.upload_files(files=['./house-price-train-data.csv'],target_path='capstoneproject/',overwrite=True, show_progress=True)
flower_data_ref = default_ds.path('capstoneproject').as_download('ex_capstoneproject')
print('reference_path = ',flower_data_ref)

In [ ]:
from azureml.core.run import Run
run = Run.get_context()
run

In [ ]:
# #define the regularization parameter for the logistic regression.
# parser = argparse.ArgumentParser()
# parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
# #define the data_folder parameter for referencing the path of the registerd datafolder.
# parser.add_argument('--data_folder', type=str, dest='data_folder', help='Data folder reference', default=0.01)
# args=parser.parse_args()
# r = args.reg
# ex_data_folder = args.data_folder
# ####

In [ ]:
# ####
# run = Run.get_context()
# estimator = SKLearn(source_directory=experiment_folder,
#                       entry_script='iris_simple_experiment.py',
#                       compute_target='local',
#                       use_docker=False,
#                       script_params = {'--reg_rate': 0.07, 
#                                   '--data_folder':flower_data_ref} 
#                               # assigned reference path value as   defined above.
#                   )
# ####

In [ ]:
default_ds

In [28]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Dataset

# Creating tabular dataset from files in datastore.
tab_dataset = Dataset.Tabular.from_delimited_files(path=(default_ds,'/capstoneproject/*.csv'))
tab_dataset.take(10).to_pandas_dataframe()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [29]:
# register tabular dataset in Workspace
tab_dataset = tab_dataset.register(workspace=ws, 
                                   name='house price train data', 
                                   description='Kaggle house price Dataset in tabular format', 
                                   tags={'format':'CSV'}, create_new_version=True)

In [30]:
print("Datasets Versions:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets Versions:
	 house price train data version 1


In [ ]:
# Get the data Using Dataset
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '9b72f9e6-56c5-4c16-991b-19c652994860'
resource_group = 'aml-quickstarts-139721'
workspace_name = 'quick-starts-ws-139721'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='house price train data')
dataset.to_pandas_dataframe()

In [ ]:
#Experiment with the Datasets
# In the training script, that trains a classification model, the tabular dataset is passed to it as input
# by reading the dataset as a Pandas Dataframe

data = run.input_datasets['house price train data'].to_pandas_dataframe()

# Use the inputs parameter of the SKLearn estimator 
# to pass the registered dataset, which is to be consumed by the training script.
inputs=[tab_dataset.as_named_input(‘flower_ds’)]